# Bimodal Noised

In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append("../../")

import os
from copy import deepcopy

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import torch
from evobench.discrete import BimodalNoised
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.seed import seed_everything
from sklearn.metrics import r2_score
from torch.utils.data import DataLoader, TensorDataset

from hell import Data, Surrogate, plot
from hell.linkage import test_lo3, test_xais


seed_everything(42)
pio.renderers.default = "notebook"

Global seed set to 42


In [2]:
benchmark = BimodalNoised(blocks=[10] * 10, multiprocessing=True, verbose=1)

data = Data(
    benchmark,
    pop_size_train=5e5,
    pop_size_val=2e4,
    pop_size_test=2e4
)

In [3]:
x_train = torch.FloatTensor(data.x_train)
x_val = torch.FloatTensor(data.x_val)
x_test = torch.FloatTensor(data.x_test)

y_train = torch.FloatTensor(data.y_train)
y_val = torch.FloatTensor(data.y_val)
y_test = torch.FloatTensor(data.y_test)


Initializing poptulation: 500000 samples
  0%|          | 1/500000 [00:00<19:45:47,  7.03it/s]
Evaluating population of 500000 solutions

  0%|          | 0/20000 [00:00<?, ?it/s]
Initializing poptulation: 20000 samples

Evaluating population of 20000 solutions

  0%|          | 0/20000 [00:00<?, ?it/s]
Initializing poptulation: 20000 samples

Evaluating population of 20000 solutions

100%|██████████| 20000/20000 [00:02<00:00, 7756.32it/s]


In [4]:
x_train *= 2
x_val *= 2
x_test *= 2

x_train -= 1
x_val -= 1
x_test -= 1

y_max = y_train.max()

y_train /= y_max
y_val /= y_max
y_test /= y_max

In [5]:
# ? sanity check

from xgboost import XGBRegressor
from sklearn.metrics import r2_score

model = XGBRegressor(n_estimators=200, nthread=8)
model.fit(x_train.cpu().numpy(), y_train.cpu().numpy())
y_pred = model.predict(x_test.cpu().numpy())
r2_score(y_test.cpu().numpy(), y_pred)

0.23261056462210183

In [12]:
train_set = TensorDataset(x_train, y_train)
val_set = TensorDataset(x_val, y_val)
test_set = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_set, batch_size=200)
val_loader = DataLoader(val_set, batch_size=200)
test_loader = DataLoader(test_set, batch_size=200)

In [13]:
surrogate = Surrogate(
    genome_size=data.benchmark.genome_size,
    n_layers=2,
    learning_rate=2e-4,
    weight_decay=1e-8
)

early_stop_callback = EarlyStopping(
   monitor="val/r2",
   min_delta=0.000,
   patience=5,
   verbose=False,
   mode="max"
)

trainer = Trainer(
    max_epochs=100,
    gpus=1,
    progress_bar_refresh_rate=50,
    callbacks=[early_stop_callback]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [14]:
trainer.fit(surrogate, train_loader, val_loader)
surrogate.eval()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params
----------------------------------------
0 | linears  | ModuleList | 16.4 K
1 | dropouts | ModuleList | 0     
2 | linear_n | Linear     | 26    
----------------------------------------
16.5 K    Trainable params
0         Non-trainable params
16.5 K    Total params
0.066     Total estimated model params size (MB)
                                                              c:\Code\academia\evo\hell-xai\env\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning:

The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.

Global seed set to 42
Epoch 0:   0%|          | 0/2600 [00:00<?, ?it/s] c:\Code\academia\evo\hell-xai\env\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWar

Surrogate(
  (linears): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=25, bias=True)
  )
  (dropouts): ModuleList(
    (0): Dropout(p=0.2, inplace=False)
    (1): Dropout(p=0.2, inplace=False)
    (2): Dropout(p=0.2, inplace=False)
  )
  (linear_n): Linear(in_features=25, out_features=1, bias=True)
)